# Instructions

The 2_flask+pandas_example folder contains the files from the previous screencast. There's nothing to do here other than to see that you can use pandas and flask together.

If you run the web app, you'll find that it still works. To run the web app:
1. In the terminal, type `cd 2_flask+pandas_example`
2. Then `python worldbank.py`
3. The data variable will print out to the terminal
4. You can open the web app in the browser as well. You can do this using the `env | grep WORLD` command in the terminal to see your WORKSPACE envirnomental variables. And from there the web address is `http://WORKSPACESPACEID-3001.WORKSPACEDOMAIN` replacing WORKSPACEID and WORKSPACEDOMAIN with your values.

http://viewc7f399f2-3001.udacity-student-workspaces.com works!!

The next step will be to use this wrangled data in a Plotly visualization. You can do this in the back-end of the web app with Plotly's Python library and Python dictionaries. And then finally, you'll need to send these dictionaries containing the visualizations to the front-end of the web app.

This is what you're going to do next.

In [ ]:
# routes.py
from worldbankapp import app
from flask import render_template
import pandas as pd
from wrangling_scripts.wrangling import data_wrangling

data = data_wrangling()

print(data)



@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html')
    
@app.route('/project-one')
def project_one():
    return render_template('project_one.html')
    

In [ ]:
# __init__.py
from flask import Flask

app = Flask(__name__)

from worldbankapp import routes


In [ ]:
# index.html
<!doctype html>

<html>
<head>
  <title>Index Page</title>
</head>

<body>
  <h1>The index.html page</h1>  
</body>
</html>


In [ ]:
# project_one.html
<!doctype html>

<html>
<head>
  <title>Project One</title>
</head>

<body>
  <h1>The project_one.html page</h1>
</body>

In [ ]:
# wrangling.py
import pandas as pd

def data_wrangling():
    df = pd.read_csv('data/API_SP.RUR.TOTL.ZS_DS2_en_csv_v2_9948275.csv', skiprows=4)

    # Filter for 1990 and 2015, top 10 economies
    df = df[['Country Name','1990', '2015']]
    countrylist = ['United States', 'China', 'Japan', 'Germany', 'United Kingdom', 'India', 'France', 'Brazil', 'Italy', 'Canada']
    df = df[df['Country Name'].isin(countrylist)]

    # melt year columns  and convert year to date time
    df_melt = df.melt(id_vars='Country Name', value_vars = ['1990', '2015'])
    df_melt.columns = ['country','year', 'variable']
    df_melt['year'] = df_melt['year'].astype('datetime64[ns]').dt.year

    # add column names
    df_melt.columns = ['country', 'year', 'percentrural']

    # prepare data into x, y lists for plotting
    df_melt.sort_values('percentrural', ascending=False, inplace=True)

    data = []
    for country in countrylist:
        x_val = df_melt[df_melt['country'] == country].year.tolist()
        y_val =  df_melt[df_melt['country'] == country].percentrural.tolist()
        data.append((country, x_val, y_val))
        print(country, x_val, y_val)
        
    return data